In [62]:
import pandas as pd
import ast

In [63]:
df_stops_total = pd.read_csv('../final-tables/stops_table.csv')
df_1980 = pd.read_csv('stops_df_1980-initial.csv')

In [64]:
df_1980.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,"Moabit, Waldstrasse",1,bus,['1']
1,1,Turmstrasse,4,bus,"['1', '16', '70']"
2,2,Turmstrasse,4,u-bahn,['9']


In [65]:
df_1980.drop(columns=['Unnamed: 0'], inplace=True)
df_1980["location"] = ""
df_1980["identifier"] = ""
df_1980["stop_id_other"] = ""
df_1980["previous_in_lines"] = ""
df_1980["stop_description"] = ""

In [66]:
df_stops_total.tail(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
11336,19761612,Zweiwinkelweg,bus,"52.532774381217585, 13.148393051743144",['80'],NaN,NaN
11337,19761613,Zwickauer Damm,bus,"52.42412841468055, 13.48547693484881",['41'],NaN,NaN
11338,19761614,zum Dorf Gatow,bus,"52.48199565682041, 13.166766484288239",['34E'],NaN,NaN


In [67]:
# Function to convert string representation of set to list
def convert_to_list(value):
    try:
        # Use ast.literal_eval to safely convert string to set
        items = ast.literal_eval(value)
        
        # Check if the result is a set, and convert it to a list
        if isinstance(items, set):
            return list(items)
        # Check if the result is an integer, and convert it to a list with a single element
        elif isinstance(items, int):
            return [items]
        # For other types, return a list with the original value
        else:
            return [value]
    except (SyntaxError, ValueError):
        # Handle the case where literal_eval fails (e.g., invalid string)
        # Try to convert the value to an integer, and return a list with a single element if successful
        try:
            return [int(value)]
        except ValueError:
            return [value]

# Apply the function to the 'in_lines' column
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(convert_to_list)
df_1980['in_lines'] = df_1980['in_lines'].apply(convert_to_list)

In [68]:
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(lambda x: ','.join(map(str, x)))
df_1980['in_lines'] = df_1980['in_lines'].apply(lambda x: ','.join(map(str, x)))

In [69]:
df_1980.head(3)

,stop_name,line_count,type,in_lines,location,identifier,stop_id_other,previous_in_lines,stop_description
0,"Moabit, Waldstrasse",1,bus,['1'],,,,,
1,Turmstrasse,4,bus,"['1', '16', '70']",,,,,
2,Turmstrasse,4,u-bahn,['9'],,,,,


In [70]:
df_stops_total.head(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19460,Adlershof,s-bahn,"52.434722222222,13.541388888889","['KBS 100a', 'KBS 100c']",Q323551,NaN
1,19461,Akazienallee,bus,"52.47849809201335, 13.607179083668733",['D'],NaN,NaN
2,19462,Alexanderplatz,bus,"52.52005246558073, 13.413957243449447","['1', '9']",NaN,NaN


In [71]:
for index, row in df_1980.iterrows():
    print(type(df_stops_total["in_lines"].values))

    # li = set(row["in_lines"].replace("'", '').replace("'", '').replace('[', '').replace(']', '').replace(" ", "").split(","))
    # print(li)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [72]:
for index, row in df_1980.iterrows():
    # Check if row["stop_name"] is in df_1960["original_name"]
    if row["stop_name"] in df_stops_total["stop_name"].values and row["in_lines"] in df_stops_total["in_lines"].values:
        # Find matching rows in df_1960
        matching_rows = df_stops_total[df_stops_total["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1980.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1980.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1980.at[index, 'stop_description'] = '+ '.join(matching_rows['stop_description'].astype(str).values)
            df_1980.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1980.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1980.at[index, 'location'] = ''
            df_1980.at[index, 'identifier'] = ''
            df_1980.at[index, 'stop_description'] = ''
            df_1980.at[index, 'stop_id_other'] = ''
            df_1980.at[index, 'previous_in_lines'] = ''


In [73]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1980['location'] = df_1980['location'].apply(split_values)
df_1980['identifier'] = df_1980['identifier'].apply(split_values)

In [74]:
df_1980.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [75]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1980 = df_1980.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [76]:
df_1980

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Achtermannstrasse,bus,['52'],,,,
1,Adalbertstrasse,bus,"['78', '75', '78E']",,,,
2,Adamstrasse,bus,"['97', '34', '13', '35']",,,,
3,Adenauerplatz,u-bahn,['7'],,,,
4,Adlershof,s-bahn,"['KBS 141', 'KBS 140', 'KBS 115', 'KBS 110']",,,,
...,...,...,...,...,...,...,...
1624,Zweiwinkelweg,bus,['80'],,,,
1625,Zwickauer Damm,bus,['41'],,,,
1626,Zwickauer Damm,u-bahn,['7'],,,,
1627,Zwieselerstrasse,bus,['94'],,,,


: 

In [19]:
df_1980.to_csv('1980-stops-combined.csv')